In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy import stats

In [48]:
# https://stackoverflow.com/questions/21546739/load-data-from-txt-with-pandas
#https://stackoverflow.com/questions/40011531/in-pandas-when-using-read-csv-how-to-assign-a-nan-to-a-value-thats-not-the



# Read data, change missing values to 'NaN'

Xtrain_raw = pd.read_csv('Xtrain.txt', sep =" ", index_col=0, dtype='float64',
                    na_values=['?','NaN'])
Xtest_raw = pd.read_csv('Xtest.txt', sep =" ", index_col=0, dtype='float64',
                    na_values=['?','NaN'])

Ytrain_raw = pd.read_csv('Ytrain.txt', sep =",", index_col=0, dtype='float64',
                    na_values=['?','NaN'])

In [49]:
Ytrain_raw.shape

(153287, 1)

In [50]:
Xtrain_raw.shape

(153287, 75)

In [51]:
Xtest_raw.shape
#Xtest_raw.head

(50000, 75)

In [52]:
#https://stackoverflow.com/questions/51207491/function-to-replace-nan-values-in-a-dataframe-with-mean-of-the-related-column
#Replace NaN with column means in Xtrain
Xtrain_filled = Xtrain_raw.apply(lambda x: x.fillna(x.mean())) 
#print(Xtrain_filled.info())

In [53]:
#Do the same for other data-sets

Ytrain_filled = Ytrain_raw.apply(lambda x: x.fillna(x.mean())) 
Xtest_filled = Xtest_raw.apply(lambda x: x.fillna(x.mean())) 

In [54]:
#Concatenate the training and test sets and check to make sure that all observations from the dataset are there

Xmerged_raw = pd.concat([Xtrain_raw, Xtest_raw])
print('X_filled shape: ', Xtest_raw.shape)
#Xmerged_raw.info()

X_filled shape:  (50000, 75)


In [55]:
Xraw_sorted = Xmerged_raw.sort_index(axis=0)
#Xraw_sorted.info()

In [57]:
#Drop the last ten rows (all -9 in test set)
Xraw_clean=Xraw_sorted.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)
#print(Xraw_clean)

In [119]:
#save as a csv file for use in R

Xraw_clean.to_csv('/Users/Jamie/Xraw_clean.csv', index = True)



In [65]:
#Do the same for the other data sets

Xtrain_clean=Xtrain_filled.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)
Xtest_clean=Xtest_filled.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)

In [66]:
Xfilled_sorted = Xraw_clean.apply(lambda x: x.fillna(x.mean()))
#Xfilled_sorted.info
#type(Xfilled_sorted)

In [67]:
#sns.pairplot(Xfilled_sorted)
#Xsample = Xfilled_sorted.sample(n = 10, random_state=1)
#type(Xsample)
#Xfilled_sorted.head

In [68]:
#Xsample_resetindex = Xsample.reset_index()
#Xsample_noindex = Xsample_resetindex.drop(['Id'], axis=1)
#Xint_ind=Xfilled_sorted['Id'].astype(str)
#TwoXcolumn=Xfilled_sorted.iloc['#A01']

#TwoXcolumn_resetindex = TwoXcolumn.reset_index()
#TwoXcolumn_noindex = TwoXcolumn_resetindex.drop(['Id'], axis=1)
#Twocolumnsdf = TwoXcolumn.copy
#Xsample_noindex
#sns.pairplot(TwoXcolumn_noindex)

In [70]:
#Merge training data sets
train_merged= Xtrain_clean.merge(Ytrain_filled, how='left', on='Id')
#print(train_merged)

In [71]:
#Remove the Id column and reset the index

train_merged_resetindex = train_merged.reset_index()


In [74]:
train_merged_noindex = train_merged_resetindex.drop(['Id'], axis=1)
#print(train_merged_noindex)

In [127]:
#Modeling [some code reused from project 2]

from sklearn.linear_model import LinearRegression, SGDRegressor,Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor

In [79]:
y = train_merged['Value'].copy()
X = train_merged.drop(['Value'], axis=1).copy()

In [80]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=32)

In [81]:
#Try Ridge Regression

In [95]:
# standardize for sample sets and Xtest
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
Xtest_clean_transformed= scaler.transform(Xtest_clean)

/Users/Jamie/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [116]:
RidgeModel = Ridge(alpha=20).fit(x_train, y_train)

In [117]:
Ridge_predictions = RidgeModel.predict(x_test)

In [118]:
mean_squared_error(y_test, Ridge_predictions, squared=False)

9.64121745271793

In [102]:

ridge_g_search = GridSearchCV(estimator = Ridge(), param_grid = {'alpha': [1.0, 2.0, 3.0, 6.0, 10, 15, 20, 30, 50, 60]}, cv = 3, n_jobs = 1, verbose = 2, return_train_score=True)

In [103]:
ridge_g_search.fit(x_train, y_train);
print(ridge_g_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ..........................................alpha=2.0; total time=   0.2s
[CV] END ..........................................alpha=2.0; total time=   0.2s
[CV] END ..........................................alpha=2.0; total time=   0.1s
[CV] END ..........................................alpha=3.0; total time=   0.2s
[CV] END ..........................................alpha=3.0; total time=   0.1s
[CV] END ..........................................alpha=3.0; total time=   0.1s
[CV] END ..........................................alpha=6.0; total time=   0.1s
[CV] END ..........................................alpha=6.0; total time=   0.1s
[CV] END .......................................

In [16]:
#Try Stochastic Gradient Descent

In [120]:
SGDtest = SGDRegressor(loss = 'huber', alpha = 0.000001, epsilon = 0).fit(x_train, y_train)

In [121]:
SGD_test_predictions = SGDtest.predict(x_test)

In [122]:
mean_squared_error(y_test, SGD_test_predictions, squared=False)

42.34306606129916

In [132]:
param_grid = {'alpha':[0.1, 0.001,],
              'learning_rate':['adaptive'],
              'max_iter':[400, 600],
              'penalty':['l2','l1','elasticnet'],
              'loss':['huber', 'squared_epsilon_insensitive', 'squared_error'],
              'eta0': [0.01, 1, 10]}

In [135]:
sgd_ = SGDRegressor(random_state = 1)
g_search = GridSearchCV(estimator = sgd_, param_grid = param_grid, cv = 3, n_jobs = 1, verbose = 2, return_train_score=True)

In [136]:
g_search.fit(x_train, y_train);
print(g_search.best_params_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l2; total time=   1.2s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l2; total time=   1.0s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l2; total time=   1.0s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l1; total time=   2.5s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l1; total time=   2.6s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l1; total time=   2.7s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=elasticnet; total time=   2.7s
[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, loss=huber, max_iter=400, penalty=elasticnet; total time=   2.6s
[CV] END alpha=0.1, eta0=

[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=huber, max_iter=600, penalty=l1; total time=   4.1s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=huber, max_iter=600, penalty=l1; total time=   7.0s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=huber, max_iter=600, penalty=elasticnet; total time=   3.6s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=huber, max_iter=600, penalty=elasticnet; total time=   3.8s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=huber, max_iter=600, penalty=elasticnet; total time=   4.0s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=400, penalty=l2; total time=  11.3s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=400, penalty=l2; total time=   7.8s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=400, penalty=l2; total time=   8.5s
[CV] END alpha=0.1, eta0=1, learning_r

[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=400, penalty=elasticnet; total time=  13.4s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=600, penalty=l2; total time=   6.1s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=600, penalty=l2; total time=   6.3s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=600, penalty=l2; total time=   6.3s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=600, penalty=l1; total time=  13.7s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=600, penalty=l1; total time=  12.1s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=600, penalty=l1; total time=  13.1s
[CV] END alpha=0.1, eta0=10, learning_rate=adaptive, loss=squared_ep

[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=l2; total time=   3.4s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=l2; total time=   3.6s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=l1; total time=   6.9s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=l1; total time=   6.0s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=l1; total time=   6.1s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=elasticnet; total time=   7.4s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=elasticnet; total time=   6.5s
[CV] END alpha=0.001, eta0=0.01, learning_rate=adaptive, loss=squared_error, max_iter=400, penalty=elasticnet; total time=   6.8s


[CV] END alpha=0.001, eta0=1, learning_rate=adaptive, loss=squared_error, max_iter=600, penalty=l1; total time=  10.4s
[CV] END alpha=0.001, eta0=1, learning_rate=adaptive, loss=squared_error, max_iter=600, penalty=elasticnet; total time=  11.6s
[CV] END alpha=0.001, eta0=1, learning_rate=adaptive, loss=squared_error, max_iter=600, penalty=elasticnet; total time=   9.2s
[CV] END alpha=0.001, eta0=1, learning_rate=adaptive, loss=squared_error, max_iter=600, penalty=elasticnet; total time=  12.0s
[CV] END alpha=0.001, eta0=10, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l2; total time=   2.4s
[CV] END alpha=0.001, eta0=10, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l2; total time=   2.7s
[CV] END alpha=0.001, eta0=10, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l2; total time=   2.4s
[CV] END alpha=0.001, eta0=10, learning_rate=adaptive, loss=huber, max_iter=400, penalty=l1; total time=   4.9s
[CV] END alpha=0.001, eta0=10, learning_rate=adaptiv

In [138]:
sgdtest = SGDRegressor(alpha = 0.001, learning_rate= 'adaptive', loss = 'squared_epsilon_insensitive', max_iter= 400 ).fit(x_train, y_train)

In [139]:
SGD_test_predictions = sgdtest.predict(x_test)

In [140]:
mean_squared_error(y_test, SGD_test_predictions, squared=False)

9.640956903016304

In [ ]:
#Try Adaboost

In [158]:
ABregr = AdaBoostRegressor(random_state=1, loss='square', n_estimators=80, learning_rate=0.01)
ABModel = ABregr.fit(x_train, y_train)

In [159]:
AB_test_predictions = ABModel.predict(x_test)

In [160]:
mean_squared_error(y_test, AB_test_predictions, squared=False)

10.08978372421733

In [ ]:
A

In [154]:
param_grid = {'random_state':[1], 'n_estimators':[80],
              'learning_rate':[.01, 0.5, 1.0, 3.0],
              'loss':['linear', 'square', 'exponential']}

In [155]:
ab_ = AdaBoostRegressor(random_state = 1)
ABg_search = GridSearchCV(estimator = ab_, param_grid = param_grid, cv = 3, n_jobs = 1, verbose = 2, return_train_score=True)

In [157]:
ABg_search.fit(x_train, y_train);
print(ABg_search.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END learning_rate=0.01, loss=linear, n_estimators=80, random_state=1; total time= 2.7min
[CV] END learning_rate=0.01, loss=linear, n_estimators=80, random_state=1; total time= 2.9min
[CV] END learning_rate=0.01, loss=linear, n_estimators=80, random_state=1; total time= 3.3min
[CV] END learning_rate=0.01, loss=square, n_estimators=80, random_state=1; total time= 2.7min
[CV] END learning_rate=0.01, loss=square, n_estimators=80, random_state=1; total time= 2.6min
[CV] END learning_rate=0.01, loss=square, n_estimators=80, random_state=1; total time= 2.7min
[CV] END learning_rate=0.01, loss=exponential, n_estimators=80, random_state=1; total time= 2.4min
[CV] END learning_rate=0.01, loss=exponential, n_estimators=80, random_state=1; total time= 2.6min
[CV] END learning_rate=0.01, loss=exponential, n_estimators=80, random_state=1; total time= 2.6min
[CV] END learning_rate=0.5, loss=linear, n_estimators=80, random_state=1; tota

In [58]:
#First a simple Linear Regression Test

LinModel = LinearRegression().fit(x_train, y_train)

In [61]:
Lin_predictions = LinModel.predict(x_test)


In [62]:
mean_squared_error(y_test, Lin_predictions, squared=False)

8.112893460636878

In [85]:
Xtest_Lin_predictions = LinModel.predict(Xtest_filled)


In [142]:
SGD_predictions = sgdtest.predict(Xtest_clean_transformed)

In [161]:
AB_predictions = ABModel.predict(Xtest_clean_transformed)

In [143]:
#Load the sample predictions
pred = pd.read_csv('pred.txt', sep =",", index_col=0, dtype= {'Value': np.float64})        

In [68]:
print(pred)

        prediction
Id                
1             10.0
6             10.0
9             10.0
12            10.0
13            10.0
...            ...
177717        10.0
177718        10.0
177726        10.0
177728        10.0
177731        10.0

[50000 rows x 1 columns]


In [89]:
sassa_pred_1 = pred.copy()
sassa_pred_1['prediction'] = Xtest_Lin_predictions

In [90]:
sassa_pred_1.to_csv('/Users/Jamie/sassa_pred_1.csv', index = True)

In [144]:
#Now Generate predictions for SGD
sassa_pred_2 = pred.copy()
sassa_pred_2['prediction'] = SGD_predictions
sassa_pred_2.to_csv('/Users/Jamie/sassa_pred_2.csv', index = True)

In [163]:
#Now Generate predictions for AdaBoost
sassa_pred_3 = pred.copy()
sassa_pred_3['prediction'] = AB_predictions
sassa_pred_3.to_csv('/Users/Jamie/sassa_pred_3.csv', index = True)

In [ ]:
#Xtrain_raw.sample
sns.pairplot(Xtrain_raw.sample(n = 2, random_state=1), height = 2)

In [4]:
mu = Ytrain_raw.mean(axis=0)
std = Ytrain_raw.std(axis=0)
#mu
std

Value    10.656328
dtype: float64

In [ ]:
#Xtrain_raw_copy= Xtrain_raw.copy()
#Xtrain_droppedNA = Xtrain_raw_copy.dropna()
Xtrain_droppedNA = Xtrain_raw.dropna()


In [ ]:
hasattr(Xtrain_droppedNA, 'sample')

In [ ]:
#scaler = StandardScaler()
Xtrain_droppedNA = scaler.fit_transform(Xtrain_droppedNA)
#df_numeric[df_numeric.columns] = scaler.fit_transform(df_numeric[df_numeric.columns])
#print(Xtrain_droppedNA['#A01'].mean())
#print(Xtrain_droppedNA['total'].std())

plt.figure(figsize=(50,40))
#cor = Xtrain_droppedNA.corr(method = 'pearson')
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

sns.pairplot(Xtrain_droppedNA.sample(n = 110, random_state=1), height = 2)